In [84]:
# Using feast to create feature store for the repo
#install feat

!pip install feast
#!pip freeze | grep feast

In [85]:
#%%bash
#cd ../
#feast init --minimal --template local features
#cd features
#touch features.py

In [32]:
import pandas as pd
import numpy as np
from pathlib import Path
import config

In [1]:
import sys

sys.path

['c:\\toolbox\\ML OPS\\End_To_End_Model_Deployment_MLOps\\notebooks',
 'C:\\toolbox\\ML OPS\\End_To_End_Model_Deployment_MLOps',
 'C:\\Users\\asokk\\.conda\\envs\\mlops\\python38.zip',
 'C:\\Users\\asokk\\.conda\\envs\\mlops\\DLLs',
 'C:\\Users\\asokk\\.conda\\envs\\mlops\\lib',
 'C:\\Users\\asokk\\.conda\\envs\\mlops',
 '',
 'C:\\Users\\asokk\\AppData\\Roaming\\Python\\Python38\\site-packages',
 'C:\\Users\\asokk\\.conda\\envs\\mlops\\lib\\site-packages',
 'C:\\Users\\asokk\\.conda\\envs\\mlops\\lib\\site-packages\\win32',
 'C:\\Users\\asokk\\.conda\\envs\\mlops\\lib\\site-packages\\win32\\lib',
 'C:\\Users\\asokk\\.conda\\envs\\mlops\\lib\\site-packages\\Pythonwin',
 'C:\\Users\\asokk\\.conda\\envs\\mlops\\lib\\site-packages\\IPython\\extensions',
 'C:\\Users\\asokk\\.ipython']

In [64]:
# Loading features to data frame
feature_path = Path(config.DATA_DIR, "credit_card_transactions.csv")

features = pd.read_csv(feature_path)

features['created_time'] = features['created_time'].astype({'created_time': 'datetime64[ms]'})

#features['created_time'] = pd.to_datetime(features['created_time'], unit='ms')

print(features.head())

features.info()

   Unnamed: 0  customer_id  Time        V1        V2        V3        V4  \
0           0            0   0.0 -1.359807 -0.072781  2.536347  1.378155   
1           1            1   0.0  1.191857  0.266151  0.166480  0.448154   
2           2            2   1.0 -1.358354 -1.340163  1.773209  0.379780   
3           3            3   1.0 -0.966272 -0.185226  1.792993 -0.863291   
4           4            4   2.0 -1.158233  0.877737  1.548718  0.403034   

         V5  Amount  Class created_time  
0 -0.338321  149.62      0   2020-09-07  
1  0.060018    2.69      0   2020-11-22  
2 -0.503198  378.66      0   2020-10-02  
3 -0.010309  123.50      0   2020-09-06  
4 -0.407193   69.99      0   2020-07-12  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 11 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   Unnamed: 0    284807 non-null  int64         
 1   customer_id   284807 n

In [65]:
# Convert to parquet format as Feast only support it thorugh offline

features.to_parquet(
    Path(config.DATA_DIR, "features.parquet"),
    compression=None,
    allow_truncated_timestamps=True
)

Feature Store


In [73]:
from feast import Entity, Feature, FeatureView, ValueType
from feast.data_source import FileSource
from google.protobuf.duration_pb2 import Duration
from datetime import datetime

define the location of the data source to feast

In [74]:
START_TIME = "2021-07-28"
feature_source = FileSource(
    path=str(Path(config.DATA_DIR, "features.parquet")),
    event_timestamp_column="created_time"
)

Nextly, define the main entity of feature data points. In our case, each data point has a unique ID.

In [75]:
feature_entity = Entity(
    name="customer_id",
    value_type=ValueType.INT64,
    description="customer id"
)

Finally, create the Feature View that loads specific features of valrious value types from the feature source for a specific period of time

In [77]:
feature_details_view = FeatureView(
    name="credit_card_transactions",
    entities=["customer_id"],
     ttl=Duration(
        seconds=(datetime.today() - datetime.strptime(START_TIME, "%Y-%m-%d")).days * 24 * 60 * 60
    ),
    features=[
        Feature(name="V1", dtype=ValueType.FLOAT),
        Feature(name="V2", dtype=ValueType.FLOAT),
        Feature(name="V3", dtype=ValueType.FLOAT),
        Feature(name="V4", dtype=ValueType.FLOAT),
        Feature(name="V5", dtype=ValueType.FLOAT), 
        Feature(name="Amount", dtype=ValueType.FLOAT),
        Feature(name="Time", dtype=ValueType.FLOAT),
        Feature(name="Class", dtype=ValueType.FLOAT)
    ],
    online=True,
    input=feature_source,
    tags={"transaction": "fradulant_transaction"},
    )

In [128]:
#%%bash
#cd ../features
#feast apply

In [188]:
%%cmd
cd ../features
feast apply

Microsoft Windows [Version 10.0.19043.1110]
(c) Microsoft Corporation. All rights reserved.

(mlops) c:\toolbox\ML OPS\End_To_End_Model_Deployment_MLOps\notebooks>cd ../features

(mlops) c:\toolbox\ML OPS\End_To_End_Model_Deployment_MLOps\features>feast apply
Registered entity ID
Registered feature view house_price_details
Deploying infrastructure for house_price_details

(mlops) c:\toolbox\ML OPS\End_To_End_Model_Deployment_MLOps\features>

In [70]:
import pandas as pd
from feast import FeatureStore
from datetime import datetime

In [71]:
customer_tran_id = [1, 2, 33]
now = datetime.now()
timestamps = [datetime(now.year, now.month, now.day)] *len(customer_tran_id)
entity_df = pd.DataFrame.from_dict({"customer_id": customer_tran_id, "event_timestamp": timestamps})
entity_df.head()

,customer_id,event_timestamp
0,1,2021-07-28
1,2,2021-07-28
2,33,2021-07-28


In [72]:
#Get historical features
store = FeatureStore(repo_path=Path(config.BASE_DIR, "features"))
training_df = store.get_historical_features(
    entity_df=entity_df,
    feature_refs=[
        "credit_card_transactions:V1", 
        "credit_card_transactions:V2"
        ],
).to_df()
training_df.head()

,event_timestamp,customer_id,credit_card_transactions__V1,credit_card_transactions__V2
0,2021-07-28 00:00:00+00:00,1,1.191857,0.266151
1,2021-07-28 00:00:00+00:00,2,-1.358354,-1.340163
2,2021-07-28 00:00:00+00:00,33,-0.529912,0.873892


Materialize - Online Store

In [ ]:
%%bash
cd ../features
CURRENT_TIME=$(date -u +"%Y-%m-%dT%H:%M:%S")
feast materialize-incremental $CURRENT_TIME

In [85]:
# Get online features
store = FeatureStore(repo_path=Path(config.BASE_DIR, "features"))
feature_vector = store.get_online_features(
    feature_refs=["credit_card_transactions:V1", "credit_card_transactions:V2"],
    entity_rows=[{"customer_id": 3}],
).to_dict()
feature_vector

FeatureViewNotFoundException: Feature view credit_card_transaction does not exist